In [1]:
# remove hash to install package
# !pip3 install owslib==0.25.0 fiona==1.8.21 geopandas==0.10.2 requests==2.28.0 folium==0.12.1

In [2]:
from owslib.wfs import WebFeatureService
import geopandas
import folium
import io
import zipfile
import pandas as pd
import os
from urllib.request import urlretrieve

## Download external data from AURIN

In [3]:
# User credential to connect with API
WFS_USERNAME = 'nyjhp'
WFS_PASSWORD= 'aFdYtPH7foNjcD58'
WFS_URL='https://adp.aurin.org.au/geoserver/wfs'

In [4]:
# Connect with API
adp_client = WebFeatureService(url=WFS_URL,username=WFS_USERNAME, password=WFS_PASSWORD, version='2.0.0')

In [5]:
def download_aurin_df(type_name, file_name):
    """
        This function downloads the dataset from AURIN API
        
        type_name: dataset identifier from the website
        file_name: output file name 
    """

    output_dir = '../data/abs'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    response = adp_client.getfeature(typename=type_name)
    out = open(f'{output_dir}/{file_name}.gml', 'wb')
    out.write(response.read())
    out.close()
    return geopandas.read_file(f'{output_dir}/{file_name}.gml')



In [6]:
# Download selected external data

pop_df = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_3:abs_regional_population_sa2_2001_2021', 
                           '2021_population_census')

sa2_bound = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_GeoLevel:sa2_2016_aust',
                        'sa2_boundaries')

poa_bound = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_GeoLevel:poa_2016_aust',
                        'poa_boundaries')

sa2_income = download_aurin_df('datasource-AU_Govt_ABS-UoM_AURIN_DB_3:abs_personal_income_total_income_distribution_sa2_2017_18', 
                        'sa2_income')

In [7]:
# Select 2021 population census

area_id = ['gml_id', 'primaryindex', 'state_code_2016', 'sa2_maincode_2016',
           'sa2_name_2016']
col_2021 = [x for x in pop_df.columns if '2021' in x or '2020_21' in x]
pop_21 = pop_df[area_id + col_2021]

In [8]:
pop_21.shape

(2292, 18)

There should be 2,310 SA2 regions.

In [9]:
# ERP refers to estimated resident population
pop_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2292 entries, 0 to 2291
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   gml_id                           2292 non-null   object 
 1   primaryindex                     2292 non-null   int64  
 2   state_code_2016                  2292 non-null   int64  
 3   sa2_maincode_2016                2292 non-null   int64  
 4   sa2_name_2016                    2292 non-null   object 
 5   erp_2021                         2292 non-null   int64  
 6   erp_change_number_2020_21        2292 non-null   int64  
 7   erp_change_per_cent_2020_21      2292 non-null   float64
 8   pop_density_2021_people_per_km2  2292 non-null   float64
 9   births_2020_21                   2288 non-null   float64
 10  deaths_2020_21                   2288 non-null   float64
 11  natural_increase_2020_21         2288 non-null   float64
 12  internal_arrivals_20

Do we want to include these regions in the data???
If so, will we include the attributes with missing data?
If yes, how?
If no, why?

In [10]:
pop_21[pop_21["births_2020_21"].isnull()]

,gml_id,primaryindex,state_code_2016,sa2_maincode_2016,sa2_name_2016,erp_2021,erp_change_number_2020_21,erp_change_per_cent_2020_21,pop_density_2021_people_per_km2,births_2020_21,deaths_2020_21,natural_increase_2020_21,internal_arrivals_2020_21,internal_departures_2020_21,net_internal_migration_2020_21,overseas_arrivals_2020_21,overseas_departures_2020_21,net_overseas_migration_2020_21
2231,abs_regional_population_sa2_2001_2021.2292,2292,9,901041004,Norfolk Island,1749,14,0.8069,45.251202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2289,abs_regional_population_sa2_2001_2021.2289,2289,9,901011001,Christmas Island,1979,15,0.7637,14.537000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2290,abs_regional_population_sa2_2001_2021.2290,2290,9,901021002,Cocos (Keeling) Islands,579,6,1.0471,42.212601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2291,abs_regional_population_sa2_2001_2021.2291,2291,9,901031003,Jervis Bay,397,2,0.5063,5.854300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
## Download Postcode to SA2 table

In [11]:
def download_url(url, filename):
    '''
        This function downloads data from the specified url.

        url: url of specified website
        filename: output file name
    '''
    output_dir = '../data/abs'
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    print(f"Begin downloading file_name data")
    output_dir = f"{output_dir}/{filename}.zip"
    urlretrieve(url, output_dir)
    print(f"Completed")

In [12]:
download_url('https://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&1270055006_CG_POSTCODE_2011_SA2_2011.zip&1270.0.55.006&Data%20Cubes&70A3CE8A2E6F9A6BCA257A29001979B2&0&July%202011&27.06.2012&Latest'
             , 'poa_sa2_lookup')

Begin downloading file_name data
Completed


In [13]:
# Open zipfile
unzip_poa_sa2 = zipfile.ZipFile('../data/abs/poa_sa2_lookup.zip') 

In [14]:
poa_to_sa2 = pd.read_excel(unzip_poa_sa2.open('1270055006_CG_POSTCODE_2011_SA2_2011.xls')
                        , sheet_name='Table 3', skiprows=5)

In [15]:
poa_to_sa2 = poa_to_sa2.dropna()

There are 2162 SA2 codes in Postcode to SA2 data, but there are 2292. Try to find newer data for poa_to_sa2?

In [16]:
poa_to_sa2["SA2_MAINCODE_2011"].unique().size

2162

In [17]:
poa_bound

,gml_id,primaryindex,objectid,poa_code_2016,poa_name_2016,area_albers_sqkm,geometry
0,poa_2016_aust.1,1,1,800,0800,3.1734,"POLYGON ((130.83450 -12.45800, 130.83390 -12.4..."
1,poa_2016_aust.2,2,2,810,0810,23.7902,"POLYGON ((130.84710 -12.37750, 130.84730 -12.3..."
2,poa_2016_aust.3,3,3,812,0812,35.8899,"POLYGON ((130.89190 -12.36880, 130.89220 -12.3..."
3,poa_2016_aust.4,4,4,815,0815,0.6381,"POLYGON ((130.87240 -12.37650, 130.87230 -12.3..."
4,poa_2016_aust.5,5,5,820,0820,39.0462,"POLYGON ((130.83500 -12.43010, 130.83510 -12.4..."
...,...,...,...,...,...,...,...
2665,poa_2016_aust.2639,2639,2639,7268,7268,162.4060,"POLYGON ((147.20120 -41.25830, 147.20130 -41.2..."
2666,poa_2016_aust.2640,2640,2640,7270,7270,265.1620,"MULTIPOLYGON (((146.82300 -41.13960, 146.82290..."
2667,poa_2016_aust.2641,2641,2641,7275,7275,340.3100,"POLYGON ((146.84830 -41.24130, 146.84850 -41.2..."
2668,poa_2016_aust.2642,2642,2642,7276,7276,5.9926,"POLYGON ((146.97820 -41.28340, 146.97820 -41.2..."


In [18]:
sa2_bound

,gml_id,primaryindex,sa2_maincode_2016,sa2_5digitcode_2016,sa2_name_2016,sa3_code_2016,sa3_name_2016,sa4_code_2016,sa4_name_2016,gccsa_code_2016,gccsa_name_2016,state_code_2016,state_name_2016,area_albers_sqkm,geometry
0,sa2_2016_aust.1,1,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58420 -35.44430, 149.58440 -35.4..."
1,sa2_2016_aust.2,2,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21900 -35.36740, 149.21800 -35.3..."
2,sa2_2016_aust.9,9,101031015,11015,Cooma Region,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6250.8748,"POLYGON ((148.60440 -36.13520, 148.60450 -36.1..."
3,sa2_2016_aust.10,10,101031016,11016,Jindabyne - Berridale,10103,Snowy Mountains,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3939.5484,"POLYGON ((148.27030 -36.46410, 148.27060 -36.4..."
4,sa2_2016_aust.11,11,101041017,11017,Batemans Bay,10104,South Coast,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,63.7074,"POLYGON ((150.23540 -35.70390, 150.23530 -35.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,sa2_2016_aust.2305,2305,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67390 -10.41570, 105.67400 -10.4..."
2306,sa2_2016_aust.2306,2306,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,"MULTIPOLYGON (((96.83050 -12.17640, 96.83050 -..."
2307,sa2_2016_aust.2307,2307,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,"MULTIPOLYGON (((150.69570 -35.18300, 150.69560..."
2308,sa2_2016_aust.2308,2308,901041004,91004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,38.6509,"MULTIPOLYGON (((167.99470 -29.04530, 167.99430..."


In [21]:
poa_bound["geometry"] = poa_bound["geometry"].to_crs(epsg=4326)
sa2_bound["geometry"] = sa2_bound["geometry"].to_crs(epsg=4326)

In [20]:
poa_bound["centroid"] = poa_bound["geometry"].centroid

/tmp/ipykernel_235/1735245261.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  poa_bound["centroid"] = poa_bound["geometry"].centroid


In [52]:
poa_bound = poa_bound[poa_bound["centroid"].notnull()]

In [46]:
sa2_bound = sa2_bound[sa2_bound["geometry"].notnull()]

In [47]:

from shapely.validation import make_valid
sa2_bound["geometry"] = sa2_bound["geometry"].apply(make_valid)

/home/aaa/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [92]:
from shapely.geometry import Point
poa_list = poa_bound["centroid"].to_list()
sa2_list = []
for point in poa_list:
    contains = sa2_bound["geometry"].contains(Point(point))
    sa2_list.append(contains.index[contains])

In [93]:
sa2_list

[Int64Index([2130], dtype='int64'),
 Int64Index([1401], dtype='int64'),
 Int64Index([1454], dtype='int64'),
 Int64Index([1401], dtype='int64'),
 Int64Index([2139], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2117], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([103], dtype='int64'),
 Int64Index([185], dtype='int64'),
 Int64Index([2308], dtype='int64'),
 Int64Index([2259], dtype='int64'),
 Int64Index([2269], dtype='int64'),
 Int64Index([1375], dtype='int64'),
 Int64Index([1943], dtype='int64'),
 Int64Index([1947], dtype='int64'),
 Int64Index([2165], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2117], dtype='int64'),
 Int64Index([1943], dtype='int64'),
 Int64Index([1946], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2120], dtype='int64'),
 Int64Index([2184], dtype='int64'),
 Int64Index([2185], dtype='int64'),
 Int64Index([2176], dtype='int64'),
 Int64Index([2175], dtype='int

In [72]:
poa_bound["sa2_area"] = pd.Series(sa2_list)

/home/aaa/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [98]:
poa_bound = poa_bound.sjoin(sa2_bound, lsuffix="sa2_area", rsuffix=sa2_bound.index)

ImportError: Spatial indexes require either `rtree` or `pygeos`. See installation instructions at https://geopandas.org/install.html